# Construção da base de notícias ESG a partir da captura do Google RSS

- Autor: Daniel Saraiva Leite - 2023
- Projeto Análise de sentimentos sobre notícias do tema ESG

In [6]:
import warnings
import pandas as pd
import datetime as dt
from noticias_google_buscador import busca_noticias_google_news
from noticias_google_buscador_esg import *
import numpy as np
from noticias_io import *

warnings.filterwarnings('ignore')

In [7]:
df_atual = le_base_noticias_bruta_para_df()
df_empresas = le_lista_empresas_para_df()

hash_noticias_ja_existentes = list(df_atual['hash'])

# processa empresas
for i, row in df_empresas[df_empresas['buscar'].str.upper() == 'SIM'].iterrows():
    
    empresa = row['empresa']
    
    aplicar_filtros= ( (not pd.isnull(row['aplicar_filtros_relevancia'])) and   row['aplicar_filtros_relevancia'].upper() == 'SIM')
    inclui_apenas_nome_empresa = ( (not pd.isnull(row['tipo_busca_exaustiva'])) and    row['tipo_busca_exaustiva'].upper() == 'SIM')
    
    atualiza = (  (not pd.isnull(row['tipo_busca_atualizacao'])) and    row['tipo_busca_atualizacao'].upper() == 'SIM' and (not pd.isnull(row['ultima_data_publicacao'])))
    ultima_data = row['ultima_data_publicacao'].date()
    
    print('\n' +  str(dt.datetime.now())  + ' Buscando empresa ' + empresa)
    dfEmpresasListadas = df_empresas
    
    df1 = busca_noticias_google_esg(empresa, atualiza=atualiza, ultima_data = ultima_data, inclui_apenas_nome_empresa = inclui_apenas_nome_empresa)
    
    if len(df1) >0:
        # ajustes
        #df1['data_publicacao'] = df1['data_publicacao'].dt.date
        df1 = ajusta_nomes_empresas_dataframe(df1)

        # cria hash
        df1['hash'] = df1.apply(lambda row: criar_hash_noticia(texto=row['titulo'], empresa=row['empresa'], titulo=row['titulo'], data=row['data_publicacao']), axis=1)
        print(str(dt.datetime.now())  + ' ... noticias encontradas: ' + str(len(df1)))
    
    if len(df1) >0:
        df1 = df1[~df1['hash'].isin(hash_noticias_ja_existentes)]
        print(str(dt.datetime.now())  + ' ... noticias novas: ' + str(len(df1)))
    
    if len(df1) > 0:
        df2 = recupera_noticias_completas(df1)
        #print(df1)
        print(str(dt.datetime.now())  + ' ... textos buscados: ' + str(len(df2)))
            
        if aplicar_filtros:
            df3 = filtra_noticias_nao_relacionadas(df2, empresa)
            if len(df3) >0:
                df4 = filtra_citacao_relevante(df3, empresa, dfEmpresasListadas )
                print(str(dt.datetime.now())  + ' ... novas noticias após filtro citaçoes: ' + str(len(df4)))
                
        else:
            df4 = df2
        
        if len(df4) >0:
            df5 = classifica_textos_coletados(df4)
            df6 = df5
            print(str(dt.datetime.now())  + ' ... novas noticias após filtros: ' + str(len(df6)))
            df_atual = le_base_noticias_bruta_para_df()
            df = pd.concat([df_atual,df6]).drop_duplicates(ignore_index=True)
            df = df.drop_duplicates(['titulo','empresa','fonte', 'texto_completo'], keep='first')
            salva_base_noticias_bruta(df)


# complementando os dados
# atualiza nome, cnpj, setor
df_noticias = le_base_noticias_bruta_para_df()
df_empresas = le_lista_empresas_para_df()
df_noticias['Nome'] = df_noticias['empresa'].map(df_empresas.set_index('empresa')['Nome'])
df_noticias['CNPJ'] = df_noticias['empresa'].map(df_empresas.set_index('empresa')['CNPJ'])
df_noticias['Razão social'] = df_noticias['empresa'].map(df_empresas.set_index('empresa')['RAZÃO SOCIAL'])
df_noticias['Código'] = df_noticias['empresa'].map(df_empresas.set_index('empresa')['Código'])
df_noticias['Setor'] = df_noticias['empresa'].map(df_empresas.set_index('empresa')['SETOR'])
salva_base_noticias_bruta(df_noticias)

# atualiza ultima data de captura
df_empresas['ultima_data_publicacao'] = df_empresas['empresa'].map(df_noticias[['empresa', 'data_publicacao']].groupby('empresa').max()['data_publicacao'] )
salva_lista_empresas(df_empresas)

print('\nCaptura concluída')


2024-03-05 21:34:44.405605 Buscando empresa camargo correa
...... buscando no google camargo+correa
2024-03-05 21:35:26.084775 ... noticias encontradas: 13
2024-03-05 21:35:26.093047 ... noticias novas: 13
2024-03-05 21:35:53.799880 ... textos buscados: 13
2024-03-05 21:35:55.955110 ... novas noticias após filtros: 13

2024-03-05 21:35:58.266362 Buscando empresa cemig
...... buscando no google cemig
2024-03-05 21:36:01.516714 ... noticias encontradas: 29
2024-03-05 21:36:01.524646 ... noticias novas: 28
2024-03-05 21:37:09.127174 ... textos buscados: 25
2024-03-05 21:37:12.313484 ... novas noticias após filtros: 25

2024-03-05 21:37:13.897398 Buscando empresa credz
...... buscando no google credz
2024-03-05 21:37:55.255489 ... noticias encontradas: 5
2024-03-05 21:37:55.266139 ... noticias novas: 5
2024-03-05 21:38:10.166687 ... textos buscados: 5
2024-03-05 21:38:10.832615 ... novas noticias após filtros: 5

2024-03-05 21:38:12.374578 Buscando empresa eletrobras
...... buscando no go